# Séance 3 : APIs et communication avec le web

Ce notebook vous guidera à travers l'utilisation des APIs web avec Python. À la fin de cette séance, vous serez capable de :
- Comprendre le fonctionnement des APIs HTTP
- Effectuer des requêtes GET et POST avec la bibliothèque `requests`
- Manipuler des données JSON
- Gérer les erreurs et les codes de statut HTTP
- Utiliser des APIs publiques (RestCountries, Open-Meteo, etc.)

---
# 1. Introduction aux APIs et HTTP

Une **API** (Application Programming Interface) permet à deux systèmes d'échanger des informations à travers un réseau (généralement Internet). La communication se fait souvent via des requêtes **HTTP**.

## Types de requêtes HTTP

| Méthode | Description | Exemple d'utilisation |
|---------|-------------|----------------------|
| **GET** | Récupérer des données | Obtenir la météo, liste de pays |
| **POST** | Envoyer des données | Créer un compte, soumettre un formulaire |
| **PUT** | Modifier des données existantes | Mettre à jour un profil |
| **DELETE** | Supprimer des données | Supprimer un article |

## Format de la réponse

Lorsqu'on interagit avec une API, on envoie une requête HTTP et l'API nous renvoie une réponse sous un format standard, souvent du **JSON** (JavaScript Object Notation).

---
# 2. La bibliothèque requests

La bibliothèque Python `requests` permet d'envoyer facilement des requêtes HTTP.

In [ ]:
# Installation de la bibliothèque (si nécessaire)
!pip install requests

In [ ]:
import requests

# Vérifier que l'installation fonctionne
print(f"requests version : {requests.__version__}")

---
# 3. Première requête GET

Commençons par une requête simple vers une API publique.

L'API **RestCountries** fournit des informations sur les pays du monde.

In [ ]:
# Vous pouvez tester cette URL dans un navigateur :
# https://restcountries.com/v3.1/all?fields=name

# Ou avec plus de champs :
# https://restcountries.com/v3.1/all?fields=name,capital,population,region

In [ ]:
import requests

# URL de l'API pour récupérer des informations sur tous les pays
url = "https://restcountries.com/v3.1/all?fields=name,capital,population,region"

# Faire la requête GET à l'API
response = requests.get(url)

# Afficher le code de statut
print(f"Code de statut : {response.status_code}")

# Vérifier que la requête a réussi (code 200)
if response.status_code == 200:
    # Récupérer les données au format JSON
    data = response.json()
    
    # Afficher quelques informations sur les premiers pays
    for country in data[:5]:  # On limite l'affichage aux 5 premiers pays
        nom = country.get('name', {}).get('common', 'N/A')
        population = country.get('population', 'N/A')
        region = country.get('region', 'N/A')
        capitale = country.get('capital', ['N/A'])[0] if country.get('capital') else 'N/A'
        
        print(f"Nom : {nom}")
        print(f"Population : {population:,}")
        print(f"Région : {region}")
        print(f"Capitale : {capitale}")
        print("-" * 30)
else:
    print(f"Erreur lors de la requête : {response.status_code}")

---
# 4. Format JSON

Le format **JSON** est une structure de données qui ressemble beaucoup à un dictionnaire Python.

Voici un exemple de réponse JSON de l'API RestCountries :

```json
{
    "name": {
        "common": "France",
        "official": "French Republic"
    },
    "capital": ["Paris"],
    "region": "Europe",
    "population": 67391582
}
```

In [ ]:
# Naviguer dans une structure JSON
exemple_json = {
    "name": {
        "common": "France",
        "official": "French Republic"
    },
    "capital": ["Paris"],
    "region": "Europe",
    "population": 67391582
}

# Accéder aux données
print(f"Nom commun : {exemple_json['name']['common']}")
print(f"Capitale : {exemple_json['capital'][0]}")
print(f"Population : {exemple_json['population']:,}")

# Utiliser .get() pour éviter les erreurs
print(f"Monnaie : {exemple_json.get('currency', 'Non disponible')}")

---
# 5. Codes de statut HTTP

Il est important de vérifier le code de statut de la réponse HTTP :

| Code | Signification |
|------|---------------|
| **200** | Succès |
| **201** | Créé avec succès (après un POST) |
| **400** | Mauvaise requête |
| **401** | Non autorisé (authentification requise) |
| **403** | Interdit (pas les permissions) |
| **404** | Ressource non trouvée |
| **429** | Trop de requêtes (rate limiting) |
| **500** | Erreur interne du serveur |

In [ ]:
import requests

def faire_requete(url):
    """Fait une requête GET et gère les codes de statut."""
    try:
        response = requests.get(url)
        
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 404:
            print("Erreur 404 : Ressource non trouvée")
        elif response.status_code == 401:
            print("Erreur 401 : Authentification requise")
        elif response.status_code == 429:
            print("Erreur 429 : Trop de requêtes, réessayez plus tard")
        else:
            print(f"Erreur {response.status_code}")
        
        return None
        
    except requests.exceptions.RequestException as e:
        print(f"Erreur de connexion : {e}")
        return None

# Test avec une URL valide
data = faire_requete("https://restcountries.com/v3.1/name/france")
if data:
    print(f"Données reçues pour : {data[0]['name']['common']}")

# Test avec une URL invalide
faire_requete("https://restcountries.com/v3.1/name/paysquinexistepas")

---
# 6. Gestion des erreurs avec try/except

In [ ]:
import requests

def requete_securisee(url, timeout=10):
    """Fait une requête GET avec gestion complète des erreurs."""
    try:
        response = requests.get(url, timeout=timeout)
        response.raise_for_status()  # Lève une exception pour les codes 4xx et 5xx
        return response.json()
        
    except requests.exceptions.Timeout:
        print("Erreur : La requête a pris trop de temps (timeout)")
    except requests.exceptions.ConnectionError:
        print("Erreur : Impossible de se connecter au serveur")
    except requests.exceptions.HTTPError as e:
        print(f"Erreur HTTP : {e}")
    except requests.exceptions.JSONDecodeError:
        print("Erreur : La réponse n'est pas au format JSON valide")
    except requests.exceptions.RequestException as e:
        print(f"Erreur de requête : {e}")
    
    return None

# Test
data = requete_securisee("https://restcountries.com/v3.1/name/germany")
if data:
    print(f"Pays trouvé : {data[0]['name']['common']}")

---
# 7. Paramètres de requête

On peut passer des paramètres dans l'URL de différentes manières.

In [ ]:
import requests

# Méthode 1 : paramètres dans l'URL
url = "https://restcountries.com/v3.1/name/france?fields=name,capital,population"
response = requests.get(url)
print(f"Méthode 1 : {response.json()[0]['name']['common']}")

# Méthode 2 : paramètres passés à requests (plus propre)
url = "https://restcountries.com/v3.1/name/france"
params = {
    "fields": "name,capital,population"
}
response = requests.get(url, params=params)
print(f"Méthode 2 : {response.json()[0]['name']['common']}")

# Afficher l'URL complète construite
print(f"URL construite : {response.url}")

---
# 8. Exemple pratique : API Météo

L'API **Open-Meteo** fournit des données météorologiques gratuitement, sans clé API.

In [ ]:
import requests

def obtenir_meteo(latitude, longitude):
    """Obtient la météo actuelle pour des coordonnées données."""
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "current_weather": True
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        meteo = data.get("current_weather", {})
        temperature = meteo.get("temperature", "N/A")
        vent_vitesse = meteo.get("windspeed", "N/A")
        code_meteo = meteo.get("weathercode", "N/A")
        
        print(f"🌡️ Température actuelle : {temperature}°C")
        print(f"💨 Vitesse du vent : {vent_vitesse} km/h")
        print(f"🌤️ Code météo : {code_meteo}")
        
        return data
        
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la requête : {e}")
        return None

# Météo à Paris (latitude: 48.8566, longitude: 2.3522)
print("=== Météo à Paris ===")
obtenir_meteo(48.8566, 2.3522)

---
# 9. Headers et authentification

Certaines APIs nécessitent des headers personnalisés ou une authentification.

In [ ]:
import requests

# Exemple avec des headers personnalisés
url = "https://api.github.com/users/octocat"

headers = {
    "Accept": "application/vnd.github.v3+json",
    "User-Agent": "MonApplication/1.0"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    user = response.json()
    print(f"Utilisateur : {user['login']}")
    print(f"Nom : {user.get('name', 'N/A')}")
    print(f"Repos publics : {user['public_repos']}")

In [ ]:
# Exemple d'authentification par token (à ne pas exécuter sans token valide)
# Remplacez YOUR_TOKEN par un vrai token pour tester

# headers = {
#     "Authorization": "Bearer YOUR_TOKEN",
#     "Accept": "application/json"
# }
# response = requests.get("https://api.exemple.com/protected", headers=headers)

print("L'authentification par token est courante pour les APIs protégées.")
print("Le token est généralement obtenu en créant un compte développeur sur le service.")

---
# 10. Requêtes POST

Les requêtes POST permettent d'envoyer des données au serveur.

In [ ]:
import requests

# Exemple avec une API de test (httpbin renvoie ce qu'on lui envoie)
url = "https://httpbin.org/post"

# Données à envoyer
donnees = {
    "nom": "Alice",
    "age": 25,
    "ville": "Paris"
}

# Envoyer une requête POST avec des données JSON
response = requests.post(url, json=donnees)

if response.status_code == 200:
    resultat = response.json()
    print("Données envoyées avec succès !")
    print(f"Le serveur a reçu : {resultat['json']}")

In [ ]:
import requests

# POST avec des données de formulaire (form data)
url = "https://httpbin.org/post"

donnees_form = {
    "username": "alice",
    "password": "secret123"
}

# Utiliser data= au lieu de json= pour envoyer comme formulaire
response = requests.post(url, data=donnees_form)

if response.status_code == 200:
    resultat = response.json()
    print("Formulaire envoyé avec succès !")
    print(f"Données de formulaire reçues : {resultat['form']}")

---
# 11. Rate Limiting et bonnes pratiques

Beaucoup d'APIs limitent le nombre de requêtes que vous pouvez faire.

In [ ]:
import requests
import time

def requete_avec_delai(urls, delai=1):
    """Fait des requêtes avec un délai entre chaque pour respecter le rate limiting."""
    resultats = []
    
    for i, url in enumerate(urls):
        print(f"Requête {i+1}/{len(urls)} : {url}")
        
        try:
            response = requests.get(url)
            response.raise_for_status()
            resultats.append(response.json())
        except requests.exceptions.RequestException as e:
            print(f"  Erreur : {e}")
            resultats.append(None)
        
        # Attendre avant la prochaine requête (sauf pour la dernière)
        if i < len(urls) - 1:
            print(f"  Attente de {delai} seconde(s)...")
            time.sleep(delai)
    
    return resultats

# Exemple avec quelques pays
urls = [
    "https://restcountries.com/v3.1/name/france?fields=name",
    "https://restcountries.com/v3.1/name/germany?fields=name",
    "https://restcountries.com/v3.1/name/spain?fields=name"
]

resultats = requete_avec_delai(urls, delai=0.5)

print("\nRésultats :")
for r in resultats:
    if r:
        print(f"  - {r[0]['name']['common']}")

---
# 12. Sauvegarder les données récupérées

In [ ]:
import requests
import json

# Récupérer des données
url = "https://restcountries.com/v3.1/region/europe?fields=name,capital,population"
response = requests.get(url)
data = response.json()

# Sauvegarder en JSON
with open('pays_europe.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print(f"Données sauvegardées : {len(data)} pays européens")

In [ ]:
import requests
import csv

# Récupérer des données
url = "https://restcountries.com/v3.1/region/europe?fields=name,capital,population"
response = requests.get(url)
data = response.json()

# Sauvegarder en CSV
with open('pays_europe.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Nom', 'Capitale', 'Population'])
    
    for pays in data:
        nom = pays.get('name', {}).get('common', 'N/A')
        capitale = pays.get('capital', ['N/A'])[0] if pays.get('capital') else 'N/A'
        population = pays.get('population', 0)
        writer.writerow([nom, capitale, population])

print(f"CSV créé avec {len(data)} lignes")

---
# 13. Exercice : Quiz des capitales

Créez un quiz qui interroge l'utilisateur sur les capitales des pays.

In [ ]:
import requests
import random

def quiz_capitales(nb_questions=5):
    """Quiz sur les capitales des pays."""
    
    # Récupérer les données des pays
    url = "https://restcountries.com/v3.1/all?fields=name,capital"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération des données : {e}")
        return
    
    # Filtrer les pays qui ont une capitale
    pays_avec_capitale = [
        p for p in data 
        if p.get('capital') and p.get('name', {}).get('common')
    ]
    
    print(f"🌍 Quiz des capitales - {nb_questions} questions")
    print("=" * 40)
    
    score = 0
    
    for i in range(nb_questions):
        # Choisir un pays au hasard
        pays = random.choice(pays_avec_capitale)
        nom_pays = pays['name']['common']
        capitale = pays['capital'][0]
        
        # Poser la question
        print(f"\nQuestion {i+1}/{nb_questions}")
        reponse = input(f"Quelle est la capitale de {nom_pays} ? ")
        
        # Vérifier la réponse (insensible à la casse)
        if reponse.lower().strip() == capitale.lower():
            print("✅ Correct !")
            score += 1
        else:
            print(f"❌ Faux ! La réponse était : {capitale}")
    
    # Afficher le score final
    print("\n" + "=" * 40)
    print(f"🏆 Score final : {score}/{nb_questions}")
    
    if score == nb_questions:
        print("Parfait ! 🎉")
    elif score >= nb_questions // 2:
        print("Bien joué ! 👍")
    else:
        print("Continuez à apprendre ! 📚")

# Lancer le quiz (décommentez pour jouer)
# quiz_capitales(5)

---
# 14. Exercice final : Application météo complète

Créez une application qui :
1. Demande à l'utilisateur d'entrer une ville et un pays
2. Utilise une API de géocodage pour obtenir les coordonnées
3. Utilise les coordonnées pour obtenir la météo

### Étape 1 : Géocodage avec Nominatim

L'API Nominatim permet de convertir un nom de lieu en coordonnées.

Testez dans un navigateur : https://nominatim.openstreetmap.org/search?q=Paris,France&format=json&limit=1

In [ ]:
import requests

def obtenir_coordonnees(ville, pays):
    """Obtient les coordonnées géographiques d'un lieu."""
    url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": f"{ville},{pays}",
        "format": "json",
        "limit": 1
    }
    
    # Nominatim exige un User-Agent
    headers = {
        "User-Agent": "FormationPython/1.0 (contact@exemple.com)"
    }
    
    # À compléter : faire la requête et extraire latitude/longitude
    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        data = response.json()
        
        if data:
            latitude = float(data[0]['lat'])
            longitude = float(data[0]['lon'])
            print(f"📍 Coordonnées trouvées : {latitude}, {longitude}")
            return latitude, longitude
        else:
            print("Lieu non trouvé")
            return None, None
            
    except requests.exceptions.RequestException as e:
        print(f"Erreur : {e}")
        return None, None

# Test
lat, lon = obtenir_coordonnees("Paris", "France")

### Étape 2 : Obtenir la météo avec Open-Meteo

In [ ]:
import requests

def obtenir_meteo_complete(latitude, longitude):
    """Obtient les informations météo détaillées."""
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "current_weather": True
    }
    
    # À compléter : faire la requête et afficher les informations météo
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        meteo = data.get("current_weather", {})
        
        print(f"\n🌡️ Température : {meteo.get('temperature', 'N/A')}°C")
        print(f"💨 Vent : {meteo.get('windspeed', 'N/A')} km/h")
        print(f"🧭 Direction du vent : {meteo.get('winddirection', 'N/A')}°")
        
        return meteo
        
    except requests.exceptions.RequestException as e:
        print(f"Erreur : {e}")
        return None

# Test avec les coordonnées de Paris
if lat and lon:
    obtenir_meteo_complete(lat, lon)

### Étape 3 : Programme principal

In [ ]:
def application_meteo():
    """Application météo interactive."""
    print("🌤️ Bienvenue dans l'application météo !")
    print("=" * 40)
    
    while True:
        # Demander le lieu
        ville = input("\nEntrez le nom de la ville : ")
        pays = input("Entrez le pays : ")
        
        # Obtenir les coordonnées
        latitude, longitude = obtenir_coordonnees(ville, pays)
        
        if latitude is not None and longitude is not None:
            # Obtenir la météo
            obtenir_meteo_complete(latitude, longitude)
        
        # Continuer ?
        choix = input("\nRechercher une autre ville ? (oui/non) : ")
        if choix.lower() != "oui":
            print("\nAu revoir ! 👋")
            break

# Lancer l'application (décommentez pour utiliser)
# application_meteo()

---
# Résumé

Dans cette séance, nous avons couvert :

1. **Introduction aux APIs** : HTTP, GET, POST
2. **Bibliothèque requests** : installation et utilisation
3. **Requêtes GET** : récupérer des données
4. **Format JSON** : structure et navigation
5. **Codes de statut** : 200, 404, 500, etc.
6. **Gestion des erreurs** : try/except avec requests
7. **Paramètres de requête** : URL et dictionnaire
8. **API météo** : exemple pratique
9. **Headers et authentification** : tokens, User-Agent
10. **Requêtes POST** : envoyer des données
11. **Rate limiting** : respecter les limites
12. **Sauvegarde** : JSON et CSV

## APIs utiles pour pratiquer

| API | Description | URL |
|-----|-------------|-----|
| RestCountries | Informations sur les pays | https://restcountries.com |
| Open-Meteo | Météo gratuite | https://open-meteo.com |
| JSONPlaceholder | API de test | https://jsonplaceholder.typicode.com |
| Cat Facts | Faits sur les chats | https://catfact.ninja |
| GitHub API | Données GitHub | https://api.github.com |